In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib
#%matplotlib inline
matplotlib.use('TkAgg')
import BPV
import pattern_manipulation as pm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gc
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
matplotlib.style.use('ggplot')
n = 70
#N=7
#minW = N*0.0005
#maxW = N*0.002
#Wsteps = 10
#Wvalues = np.arange(minW,maxW,(maxW-minW)/Wsteps)

Wmin = 0.001
Wmax = 0.01
Wnsteps = 10
Wstep = (Wmax - Wmin)/Wnsteps
Wrange = np.arange(Wmin,Wmax,Wstep)

Nmin = 3
Nmax = 10
Nnsteps = 10
Nstep = (Nmax - Nmin)/Nnsteps
Nrange = np.arange(Nmin,Nmax,Nstep, dtype='int')

nproblems = Wnsteps*Nnsteps


Hprbls1 = {}
Wprbls1 = {}
Hprbls2 = {}
Wprbls2 = {}

#colnames = []
#for W in Wvalues:
#    colnames.append('W=%.3f' % (W))
df = BPV.Data()
df.read_csv("pixel.dist.csv",False)

df.df.sort_index(by="p",inplace=True,ascending=True)
df.df.set_index(pd.Index([j for j in range(len(df.df))]), inplace=True)
dfh1 = df.data_head(n)
dfh1.df.set_index(pd.Index([j for j in range(len(dfh1.df))]), inplace=True)

df.df.sort_index(by="p",inplace=True,ascending=False)
df.df.set_index(pd.Index([j for j in range(len(df.df))]), inplace=True)
dfh2 = df.data_head(n)
dfh2.df.set_index(pd.Index([j for j in range(len(dfh2.df))]), inplace=True)
dfh2.df.sort_index(by="p",inplace=True,ascending=True)

p1 = dfh1.df['p']
p2 = dfh2.df['p']
print(p1.var(), p2.var())

#dfh1.df
#Wvalues
#colnames

1.60522640976e-05 0.00520544414612


In [3]:
fig = plt.figure()
plt.plot(p1,'r')
plt.plot(p2,'b')
plt.yscale('log')
plt.show()

In [53]:
firstiter = True
it = 1
#for W,col in zip(Wvalues,colnames):
for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    W = Wrange[i]
    N = Nrange[j]    
    print("\n\n-----------------------\nITERATION {0}/{1} - W = {2} N = {3} - i = {4} j = {5}\n"\
          .format(it,nproblems,W,N,i,j))
    
    #prbl_pulp = BPV.BPV("pulp",dfh1,N,W,time_solver=True)
    #prbl_pulp.solve()
    #prbl_pulp.pprint_solution()

    prbl_decH = BPV.BPV("decgraphH",dfh1,N,W,time_solver=True)
    prbl_decH.solve()
    #cProfile.run('prbl.solve()',sort=1)
    prbl_decH.pprint_solution()
    del prbl_decH.predecessor
    Hprbls1[i,j] = prbl_decH
    
    prbl_decW = BPV.BPV("decgraphW",dfh1,N,W,time_solver=True)
    prbl_decW.solve()
    #cProfile.run('prbl.solve()',sort=1)
    prbl_decW.pprint_solution()
    del prbl_decW.predecessor
    Wprbls1[i,j] = prbl_decW
    
    prbl_decH = BPV.BPV("decgraphH",dfh2,N,W,time_solver=True)
    prbl_decH.solve()
    #cProfile.run('prbl.solve()',sort=1)
    prbl_decH.pprint_solution()
    del prbl_decH.predecessor
    Hprbls2[i,j] = prbl_decH
    
    prbl_decW = BPV.BPV("decgraphW",dfh2,N,W,time_solver=True)
    prbl_decW.solve()
    #cProfile.run('prbl.solve()',sort=1)
    prbl_decW.pprint_solution()
    del prbl_decW.predecessor
    Wprbls2[i,j] = prbl_decW
    
    gc.collect()
    it+=1




-----------------------
ITERATION 1/4 - W = 0.001 N = 3 - i = 0 j = 0

Problem not solved
Problem not solved


KeyboardInterrupt: 

In [51]:

W = Wrange[0]
N = Nrange[0]
prbl_decH = BPV.BPV("decgraphH",dfh1,N,W,time_solver=True)
prbl_decH.solve()
#cProfile.run('prbl.solve()',sort=1)
prbl_decH.pprint_solution()
W,N

Problem not solved


(0.001, 3)

In [20]:
#SAVE 

fH1 = open("/home/renato/tesi/code/data/plot2.Hprbls1",r'wb')
fW1 = open("/home/renato/tesi/code/data/plot2.Wprbls1",r'wb')
fH2 = open("/home/renato/tesi/code/data/plot2.Hprbls2",r'wb')
fW2 = open("/home/renato/tesi/code/data/plot2.Wprbls2",r'wb')
pickle.dump(Hprbls1,fH1)
pickle.dump(Wprbls1,fW1)
pickle.dump(Hprbls2,fH2)
pickle.dump(Wprbls2,fW2)


In [54]:
#LOAD

fH1 = open("/home/renato/tesi/code/data/plot2.Hprbls1",r'rb')
fW1 = open("/home/renato/tesi/code/data/plot2.Wprbls1",r'rb')
fH2 = open("/home/renato/tesi/code/data/plot2.Hprbls2",r'rb')
fW2 = open("/home/renato/tesi/code/data/plot2.Wprbls2",r'rb')
Hprbls1 = pickle.load(fH1)
Wprbls1 = pickle.load(fW1)
Hprbls2 = pickle.load(fH2)
Wprbls2 = pickle.load(fW2)


In [55]:
entropies1 = np.zeros((Wnsteps, Nnsteps))
#rates1 = np.zeros((Wnsteps, Nnsteps))
#cardinalities1 = np.zeros((Wnsteps, Nnsteps), dtype='int')
times1 = np.zeros((Wnsteps, Nnsteps))
nnodes1 = np.zeros((Wnsteps,Nnsteps))
indexes1 = {}

entropies2 = np.zeros((Wnsteps, Nnsteps))
#rates2 = np.zeros((Wnsteps, Nnsteps))
#cardinalities2 = np.zeros((Wnsteps, Nnsteps), dtype='int')
times2 = np.zeros((Wnsteps, Nnsteps))
nnodes2 = np.zeros((Wnsteps,Nnsteps))
indexes2 = {}

it = 1
for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    #entropies1[i,j] = Hprbls1[i,j].solution_entropy
    print(i,j)
    Wprbls1[i,j].pprint_solution()

0 0
Problem not solved
0 1
Problem not solved
1 0
Problem not solved
1 1
Problem not solved


In [13]:
matplotlib.style.use('ggplot')
#plt.rcdefaults()

label_size = 16
ms = 2
_dpi = 250
#plt.ylim((-100,int(10e5)))
ax = plt.gca()
ax.set_yscale('log')
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.plot(Wvalues[:len(Hnnodes)],Hnnodes,'rx-',label='H solver',markeredgewidth=ms)
plt.plot(Wvalues[:len(Wnnodes)],Wnnodes,'gx-',label='W solver',markeredgewidth=ms)
plt.legend(loc=2)
plt.xlabel(r'$W$',fontsize=label_size)
plt.ylabel(r'number of nodes',rotation='vertical', fontsize=label_size, horizontalalignment='right')
#plt.savefig("/home/renato/tesi/testogit/img/nnodesW.pdf",dpi=_dpi,bbox_inches='tight')
plt.show()

In [10]:
Hnnodes = []
Hvisitlist = {}
Wnnodes = []
Wvisitlist = {}
for k,v in Hprbls.items():
    Hnnodes.append(v.nnodes)
    Hvisitlist[k] = v.decgraph_len_visitlist
Hnnodes.sort()

for k,v in Wprbls.items():
    Wnnodes.append(v.nnodes)
    Wvisitlist[k] = v.decgraph_len_visitlist
Wnnodes.sort()
#print(Hnnodes,Wnnodes)


In [14]:
#LENVISITLIST 1

fig = plt.figure(figsize=(10,10))
ax = plt.gca()
_dpi = 200
fig.dpi = _dpi
label_size = 16
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
trange = np.arange(0,20,1,'int')

plt.subplot(2, 1, 1)
maxlen=0
for vl in Wvisitlist.values():
    maxlen = max(maxlen, max(vl))

    
plt.ylim((0,maxlen*1.1))
plt.yscale('linear')
plt.plot(trange,2**trange,'k',label='$2^k$')
#plt.plot(trange,trange**3,'k--',label='$k^3$')
for W,vl in Hvisitlist.items():
    #ax.set_yscale('log')
    plt.plot(vl,label='$'+str(W)+'$')
    plt.legend(loc=1)
plt.xlabel(r'$k$',fontsize=label_size)
plt.ylabel(r'$|M_k^{\mathcal{H}}|$',rotation='vertical', fontsize=label_size, horizontalalignment='right')

plt.subplot(2, 1, 2)
maxlen=0


for vl in Wvisitlist.values():
    maxlen = max(maxlen, max(vl))

plt.ylim((0,maxlen*1.1))
plt.yscale('linear')
plt.plot(trange,2**trange,'k',label='$2^k$')
#plt.plot(trange,trange**3,'k--',label='$k^3$')
for W,vl in Wvisitlist.items():
    #ax.set_yscale('log')
    plt.plot(vl, label='$'+str(W)+'$')
    plt.legend(loc=1)
plt.xlabel(r'$k$',fontsize=label_size)
plt.ylabel(r'$|M_k^{\mathcal{W}}|$',rotation='vertical', fontsize=label_size, horizontalalignment='right')

plt.tight_layout()
#plt.savefig("/home/renato/tesi/testogit/img/bpv-lenvisitlist.pdf",dpi=_dpi,bbox_inches='tight')
plt.show()